<div class="cite2c-biblio"></div><img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> INF-493 - Machine Learning</h1>
    <h1> Tarea 1 - Metodos Lineales para Regresión </h1>
</center>
<p>
<br><center>_Javier Reyes_<strong> - </strong>_javier.reyes.12@sansano.usm.cl_<strong> - </strong>_201273524-6_ </center>
<br><center>_Marco Salinas_<strong> - </strong>_marco.salinas.12@sansano.usm.cl_<strong> - </strong>_201273589-0_ </center>
</p>

# Introducción


#  1 Regresión Lineal Ordinaria (LSS)


<p  style="text-align: justify;"> </p>

<p  style="text-align: justify;"> 
    **(a)** Construya un dataframe con los datos a analizar descargándolos desde la plataforma como se indicó.
Explique por qué se realiza la línea 4.
</p>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("kc_house_data.csv")
df.drop(['id','date','zipcode',],axis=1,inplace=True)
df.head()

<p  style="text-align: justify;">
    La función _drop_ utilizada en el código anterior, elimina las columnas "id", "date" y "zipcode" del dataframe, que anteriormente habiamos leido. Esto se realiza debido a que dichos atributos no son relevantes para el estudio del dataframe.
</p>

<p  style="text-align: justify;"> 
    **(b)** Describa brevemente el dataset a utilizar.
</p>

In [ ]:
print(df.shape)
print("------------------------------------------")
df.info()
df.describe()

<p  style="text-align: justify;">
    El dataset posee $21613$ datos, los cuales poseen $18$ atributos cada uno donde describen las distintas caracteristicas de las casa, por ejemplo, el precio de venta de la casa, la cantidad de habitaciones y baños, la cantidad de metros cuadrados construidos, y los del terreno. Además, de otras especificaciones tecnicas como el año de construcción, el año de la ultima renovacion, el promedio de metros cuadrados construidos y de terreno de las 15 casas más cercanas. 
</p>

<p  style="text-align: justify;">
    Todos estos datos nos ayudarian a tomar decisiones sobre cual casa se deberia comprar.
</p>

<p  style="text-align: justify;">
    **(c)** Normalice los datos antes de trabajar y aplique una transformación adecuada a la variable a predecir.
Explique la importancia/conveniencia de realizar estas dos operaciones.
</p>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['price'] = np.log(df['price'])

<p  style="text-align: justify;">
    **(d)** Realice una regresión lineal de mínimos cuadrados básica. Explique la importancia/conveniencia del
paso 4 y los argumentos que se deben entregar a la función que implementa la regresión lineal.
</p>

In [ ]:
import sklearn.linear_model as lm
X = df_scaled.iloc[:,1:] #use .ix instead, in older pandas version
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['price']
#mascara estatica con el 70% de los datos
mascara = np.zeros(len(X))
limit = int(len(X)*0.7)
mascara[:limit] = 1
istrain = mascara== 1
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

<p  style="text-align: justify;">
    **(e)** Construya una tabla con los pesos y Z-score correspondientes a cada predictor (variable). ¿Qué variables
están más correlacionadas con la respuesta? Si usáramos un nivel de significación del 5%. ¿Qué es lo
que observa y cuál puede ser la causa?
</p>


In [ ]:
from scipy import stats
w = linreg.coef_ #pesos predictor
w2 = linreg(alpha = 0.05)
zscoreX = sum(stats.zscore(Xtrain, axis=1)) #zscore predictor
wz = np.empty((18,2))
for i in range(0,18):
    wz[i][0] = linreg.coef_[i]
    wz[i][1] = zscore[i]
df = pd.DataFrame(wz, columns = ["Peso","Z-score"])
df.index = ["bedrooms","bathrooms","sqft_living","sqft_lot","floors","waterfront","view","condition",
"grade","sqft_above","sqft_basement","yr_built","yr_renovated","lat","long","sqft_living15","sqft_lot15","intercept"] 
slope, intercept, r_value, p_value, std_err = stats.linregress(Xtrain,ytrain)
df

<p  style="text-align: justify;">
    Las variables más correlocionadas son: sqft_above, sqft_basement y lat
</p>


<p  style="text-align: justify;">
    **(f)** Proponga un método para corregir lo observado (Hint: inspírese en los métodos de feature engineering
de las siguiente secciones). Verifíquelo mediante los Z-score presentados en la pregunta *(e)*.
</p>

<p  style="text-align: justify;">
    **(g)** Estime el error de predicción del modelo usando validación cruzada con un número de folds igual a $K
= 5$ y $K = 10$. Recuerde que para que la estimación sea razonable, en cada configuración (_fold_) debería
reajustar los pesos del modelo. Mida el error real del modelo sobre el conjunto de pruebas, compare y
concluya.
</p>

In [ ]:
yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
mse_cv = 0
for train, val in kf.split(Xm):
    linreg = lm.LinearRegression(fit_intercept = False)
    linreg.fit(Xm[train], ym[train])
    yhat_val = linreg.predict(Xm[val])
    mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
    mse_cv += mse_fold
mse_cv = mse_cv / 10

<p  style="text-align: justify;">
    **(h)** Mida los errores de predicción para cada dato de entrenamiento. Utilizando un "quantile-quantile plot"
determine si es razonable la hipótesis de normalidad sobre los residuos del modelo.
</p>

#  2 Selección de Atributos


<p  style="text-align: justify;">
    **(a)** Construya una función que implemente Forward Step-wise Selection (FSS). Es decir, partiendo con un
modelo sin predictores (variables), agregue un predictor a la vez, re-ajustando el modelo de regresión
en cada paso. Para seleccionar localmente una variable, proponga/implemente un criterio distinto al
utilizado en el código de ejemplo. Construya un gráfico que muestre el error de entrenamiento y el error
de pruebas como función del número de variables en el modelo. Ordene el eje x de menor a mayor.

</p>

In [ ]:
def fss(x, y, names_x, k = 10000):
    p = x.shape[1]-1
    k = min(p, k)
    names_x = np.array(names_x)
    remaining = range(0, p)
    selected = [p]
    current_score = best_new_score = 0.0
    while remaining and len(selected)<=k :
        score_candidates = []
        for candidate in remaining:
            model = lm.LinearRegression(fit_intercept=False)
            indexes = selected + [candidate]
            x_train = x[:,indexes]
            predictions_train = model.fit(x_train, y).predict(x_train)
            residuals_train = predictions_train - y
            mse_candidate = np.mean(np.power(residuals_train, 2))
            score_candidates.append((mse_candidate, candidate))
        score_candidates.sort()
        score_candidates[:] = score_candidates[::-1]
        best_new_score, best_candidate = score_candidates.pop()
        remaining.remove(best_candidate)
        selected.append(best_candidate)
        print ("selected = %s ..."%names_x[best_candidate])
        print ("totalvars=%d, mse = %f"%(len(indexes),best_new_score))
    return selected
names_regressors = X.columns[:-1] #without intercept
fss(Xm,ym,names_regressors)

<p  style="text-align: justify;">

</p>

#  3 Regularización

<p  style="text-align: justify;">
    **(a)** Ajuste un modelo lineal utilizando "Ridge Regression", es decir, regularizando con la norma $l_2$. Utilice
valores del parámetro de regularización $\lambda$ en el rango $[10^7, 10^1]$, variando si estima conveniente.
Construya un gráfico que muestre los coeficientes obtenidos como función del parámetro de regularización. Describa lo que observa. (WARNING: Note que la línea 3 y el primer argumento en la línea 9
son críticos).

</p>

In [ ]:
from sklearn.linear_model import Ridge
import matplotlib.pylab as plt
X2 = X.drop('intercept', axis=1,inplace=False)
Xtrain = X2[istrain]
ytrain = y[istrain]
names_regressors = X2.columns
alphas_ = np.logspace(7,1,base=10)
coefs = []
model = Ridge(fit_intercept=True,solver='svd')
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
plt.figure(figsize=(18,9))
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
    plt.plot(alphas_, y_arr, label=label)
    plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.title('Regularization Path RIDGE')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

<p  style="text-align: justify;">
    **(b)** Ajuste un modelo lineal utilizando el método "Lasso", es decir, regularizando con la norma $l_1$. Utilice
valores del parámetro de regularización $\lambda$ en el rango $[10^0, 10^-3]$. Para obtener el código, modifique
las líneas 7 y 9 del ejemplo anterior. Construya un gráfico que muestre los coeficientes obtenidos
como función del parámetro de regularización. Describa lo que observa. ¿Es más efectivo Lasso para
seleccionar atributos?.

</p>

In [ ]:
from sklearn.linear_model import Ridge
import matplotlib.pylab as plt
X2 = X.drop('intercept', axis=1,inplace=False)
Xtrain = X2[istrain]
ytrain = y[istrain]
names_regressors = X2.columns
alphas_ = np.logspace(0,-3,base=10)
coefs = []
model = Lasso(fit_intercept=True)
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
plt.figure(figsize=(18,9))
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
    plt.plot(alphas_, y_arr, label=label)
    plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.title('Regularization Path Lasso')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

<p  style="text-align: justify;">
    **(c)** Escogiendo uno de los dos métodos regularizadores anteriores, especificando el porqué, construya un
gráfico que muestre el error de entrenamiento y el error de pruebas como función del parámetro de
regularizacion. Discuta lo que observa.
</p>

In [ ]:
Xtest = X2[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
alphas_ = np.logspace(0,-3,base=10) #np.logspace(7,1,base=10)
coefs = []
model =  #Ridge(fit_intercept=True,solver='svd') #Lasso(fit_intercept=True) 
mse_test = []
mse_train = []
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    yhat_train = model.predict(Xtrain)
    yhat_test = model.predict(Xtest)
    mse_train.append(np.mean(np.power(yhat_train - ytrain, 2)))
    mse_test.append(np.mean(np.power(yhat_test - ytest, 2)))
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error ridge')
ax.plot(alphas_,mse_test,label='test error ridge')
plt.legend(loc=1)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

<p  style="text-align: justify;">
    **(d)** Estime el valor del parámetro de regularización en **alguno** de los modelos anteriores haciendo uso de
la técnica validación cruzada.

</p>

In [ ]:
def MSE(y,yhat): return np.mean(np.power(y-yhat,2))
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
best_cv_mse = float("inf")
model = #choose it
alphas_ = #alphas to evaluate
for a in alphas_:
    model.set_params(alpha=a)
    mse_list_k10 = [MSE(model.fit(Xm[train], ym[train]).predict(Xm[val]), ym[val]) for train, val in kf.split(Xm)]
    if np.mean(mse_list_k10) < best_cv_mse:
        best_cv_mse = np.mean(mse_list_k10)
        best_alpha = a
        print "BEST PARAMETER=%f, MSE(CV)=%f"%(best_alpha,best_cv_mse)

#  4 Drift

<p  style="text-align: justify;">
    En esta sección se presentarán dos muestras del dataframe utilizado en la actividades anteriores, donde cada
una de estas tiene una propiedad distinta ya que son muestreadas en función del valor a predecir (logaritmo
del precio de la casa). Por una parte se tiene una peque~na muestra A, la cual es extraída directamente de
los datos con los que se trabaja (manteniendo la distribución de esta) y la muestra B, es generada con el
propósito de que en cada intervalo del rango de valores haya la misma cantidad de datos aproximadamente
(simulando una distribución uniforme). El objetivo es familiarizarse con el concepto de _Transfer Learning_.

</p>

In [ ]:
df_A = df_scaled.sample(1000,random_state=11)

frames = []
valor = df_scaled.price
length = 0.3
for z in np.arange(int(np.min(valor)),int(np.max(valor))+1,length):
    #un maximo de 100 datos por intervalo
    aux = df_scaled[(df_scaled.price >= z) & (df_scaled.price < z+length)].head(100)
    frames.append(aux)
df_B = pd.concat(frames).sample(1000,random_state=11) #crea el dataframe

<p  style="text-align: justify;">
    **(a)** Cree el conjunto de entrenamiento y otro de validación para trabajar cada muestra mediante la técnica
de _hold out validation_.

</p>

In [ ]:
X_A = df_A.iloc[:,1:].values
y_A = df_A.price
X_B = df_B.iloc[:,1:].values
y_B = df_B.price
from sklearn.model_selection import train_test_split
Xtrain_A,Xval_A,ytrain_A,yval_A = train_test_split(X_A, y_A, test_size=0.3, random_state=42)
Xtrain_B,Xval_B,ytrain_B,yval_B = train_test_split(X_B, y_B, test_size=0.3, random_state=42)

<p  style="text-align: justify;">
    **(b)** Cree el conjunto de entrenamiento y otro de validación para trabajar cada muestra mediante la técnica
de _hold out validation_.

</p>

<p  style="text-align: justify;">
    Respuesta
</p>

<p  style="text-align: justify;">
    **(c)** Cree el conjunto de entrenamiento y otro de validación para trabajar cada muestra mediante la técnica
de _hold out validation_.

</p>

<p  style="text-align: justify;">
    Respuesta
</p>

#  5 Detectar enfermedades cardíacas

<p  style="text-align: justify;">
    En el área de la salud, diagnosticar a una persona de una enfermedad de forma rápida y correcta puede llegar
a salvarle la vida. Los encargados de realizar estos diagnósticos, son médicos que, observando exámenes y
ciertos indicadores, pueden concluir qué enfermedad presenta el paciente. Si el medico se llegase a equivocar,
aparte de que el paciente pueda perder la vida, el medico podría ser demandado por negligencia arriesgando
a~nos de cárcel o pagar sumas de dinero considerable, es por estas razones que es importante no cometer
errores.
Pongámonos en el contexto de que usted es contratado para generar un modelo que prediga si es que un
paciente presenta una enfermedad cardiaca a partir de ciertos indicadores, tales como la edad, sexo, presión sanguínea, nivel de glicemia, etc. Los datos para trabajar junto a su documentación pueden ser descargados
ejecutando los siguientes comandos en un terminal (sistemas UNIX)
</p>

- wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat
- wget https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.doc

<p  style="text-align: justify;">
    Como ayuda se le indica que la variable de máximo ritmo cardíaco alcanzado (_maximum heart rate achieved_)
es un buen indicador de detección de enfermedades cardíacas. Por lo que el objetivo es predecir el comportamiento
de esta variable en función de las otras, y con esto detectar qué tan distante es el valor real al valor
predecido para así luego detectar los posibles _outliers_ (enfermos), que en sí corresponden a pacientes que
tienen un comportamiento anormal al resto.

</p>

<p  style="text-align: justify;">
   **(a)** Lea el archivo de datos, cárguelos en un dataframe o matriz, luego divida el dataframe en dos, un
dataframe de entrenamiento (70% Datos) y un dataframe de prueba (30% Datos).

</p>

In [ ]:
headers = ["age","sex","chest_pain","blood_p","serum","blood_s","electro","max_heart","angina"
           ,"oldpeak","slope","vessel","thal","normal"]


df = pd.read_table("heart.dat", header=None, names=headers, sep=" ", engine="python")
train, test = train_test_split(df, test_size=0.3)

<p  style="text-align: justify;">
   **(b)** Realice una regresión lineal y defina usted una frontera de decisión (umbral) para poder determinar si
es que estamos en presencia o no de una enfermedad cardíaca. Mida su desempeño con ambos conjuntos
de datos.

</p>

In [ ]:



from sklearn.metrics import accuracy_score
print ("Score: "%(accuracy_score(y_outlier,y_predict_outlier))

<div id='obj' />
# Referencias

<ul style="text-align: justify;">
    <li>https://www.kaggle.com/harlfoxem/housesalesprediction/discussion/23194</li>
    <li>https://rstudio-pubs-static.s3.amazonaws.com/155304_cc51f448116744069664b35e7762999f.html</li>
    <li>http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html</li>
    <li>https://fisicamatematic.wordpress.com/2011/11/04/minimos-cuadrados</li>
    <li></li>
    <li></li>
</ul>

<div id='obj' />

